In [ ]:

from kiteconnect import KiteTicker, KiteConnect
import datetime as dt
import sys
import pandas as pd
import os
import time
import numpy as np
import pandas_ta as ta
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [ ]:
cwd = os.chdir(r"C:\Users\Admin\stocksdata")
#generate trading session
access_token = open("access_token.txt",'r').read()
key_secret = open("api_key.txt",'r').read().split()
kite = KiteConnect(api_key=key_secret[0])
kite.set_access_token(access_token)

In [ ]:
instrument_dump = kite.instruments("NSE") 
instrument_df = pd.DataFrame(instrument_dump)

def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [ ]:
instrument_dump_nfo = kite.instruments("NFO") 
instrument_df_nfo = pd.DataFrame(instrument_dump_nfo)
fno=instrument_df_nfo["name"].unique()

In [ ]:
def OHLC_data(ticker):
 
    data = fetchOHLC(ticker,"5minute",100)
    ohlc_dict = {
        'open':'first',
        'high':'max',
        'low':'min',
        'close':'last',
        'volume':'sum'
        }

    #data1 = data.resample('W').agg(ohlc_dict)
   # print(data1)
    #pd.concat([data,data1], axis=0, join='inner')
    #data1 = data1.copy()
    data["Entity Name"] = ticker
    
    return(data)

In [ ]:
data =OHLC_data("GAIL")
data

In [ ]:
sbi_open = data[["close"]]
sbi_open.plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sbi_val = sbi_open.values

In [ ]:
plt.plot(sbi_val)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
normalizer = MinMaxScaler(feature_range=(0,1))
data_scaled = normalizer.fit_transform(np.array(sbi_val).reshape(-1,1))

In [ ]:
len(data_scaled), len(sbi_val)

In [ ]:
# Splitting the data into train & test
train_size = int(len(data_scaled)*0.7)
test_size = len(data_scaled) - train_size

In [ ]:
train_size, test_size

In [ ]:
data_scaled.shape

In [ ]:
data_scaled[0:train_size].shape

In [ ]:
 data_scaled[train_size:len(data_scaled)].shape

In [ ]:
# splitting data into train & test
data_train, data_test = data_scaled[0:train_size], data_scaled[train_size:len(data_scaled)]

In [ ]:
data_train.shape, data_test.shape

In [ ]:
len(data_train) - 100

In [ ]:
# Creating a dataset in timeseries for LSTM model

def create_data(dataset, step):
    Xtrain,ytrain = [],[]
    for i in range(len(dataset) - step - 1):
        a = dataset[i:(i+step),0]
        Xtrain.append(a)
        ytrain.append(dataset[i + step, 0])
        
    return np.array(Xtrain), np.array(ytrain)

In [ ]:
# Taking 100 days price as one record for training

time_stamp = 200

X_train, y_train = create_data(data_train, time_stamp)
X_test, y_test = create_data(data_test, time_stamp)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.shape[0],X_train.shape[1]

In [ ]:
X_train.reshape(X_train.shape[0], X_train.shape[1],1)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

In [ ]:
# Reshaping the datato fit into LSTM model
X_train.shape, X_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

In [ ]:
# Creating LSTM model using keras
model = Sequential()
model.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1],1)))
model.add(LSTM(units = 100, return_sequences = True))
model.add(LSTM(units = 100, return_sequences = True))
model.add(GRU(units = 100))
model.add(Dense(units = 1, activation = "linear"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "mean_squared_error", optimizer = "adam", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100)

In [ ]:
loss = model.history.history["loss"]
plt.plot(loss)

In [ ]:
# predicting on train & test data
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# Inverse transform to get actual value
train_predict = normalizer.inverse_transform(train_predict)

test_predict = normalizer.inverse_transform(test_predict)

In [ ]:
plt.plot(normalizer.inverse_transform(data_train),color = "b", label = "actual")
plt.plot(train_predict, color = 'r', label = "train")
plt.plot(test_predict, color = 'g',label = "test")
plt.legend()
plt.show()